## --- Day 14: Regolith Reservoir ---

The distress signal leads you to a giant waterfall! Actually, hang on - the signal seems like it's coming from the waterfall itself, and that doesn't make any sense. However, you do notice a little path that leads behind the waterfall.

Correction: the distress signal leads you behind a giant waterfall! There seems to be a large cave system here, and the signal definitely leads further inside.

As you begin to make your way deeper underground, you feel the ground rumble for a moment. Sand begins pouring into the cave! If you don't quickly figure out where the sand is going, you could quickly become trapped!

Fortunately, your familiarity with analyzing the path of falling material will come in handy here. You scan a two-dimensional vertical slice of the cave above you (your puzzle input) and discover that it is mostly air with structures made of rock.

Your scan traces the path of each solid rock structure and reports the x,y coordinates that form the shape of the path, where x represents distance to the right and y represents distance down. Each path appears as a single line of text in your scan. After the first point of each path, each point indicates the end of a straight horizontal or vertical line to be drawn from the previous point. For example:
```
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
```
This scan means that there are two paths of rock; the first path consists of two straight lines, and the second path consists of three straight lines. (Specifically, the first path consists of a line of rock from 498,4 through 498,6 and another line of rock from 498,6 through 496,6.)

The sand is pouring into the cave from point 500,0.

Drawing rock as #, air as ., and the source of the sand as +, this becomes:

```
  4     5  5
  9     0  0
  4     0  3
0 ......+...
1 ..........
2 ..........
3 ..........
4 ....#...##
5 ....#...#.
6 ..###...#.
7 ........#.
8 ........#.
9 #########.
```
Sand is produced one unit at a time, and the next unit of sand is not produced until the previous unit of sand comes to rest. A unit of sand is large enough to fill one tile of air in your scan.

A unit of sand always falls down one step if possible. If the tile immediately below is blocked (by rock or sand), the unit of sand attempts to instead move diagonally one step down and to the left. If that tile is blocked, the unit of sand attempts to instead move diagonally one step down and to the right. Sand keeps moving as long as it is able to do so, at each step trying to move down, then down-left, then down-right. If all three possible destinations are blocked, the unit of sand comes to rest and no longer moves, at which point the next unit of sand is created back at the source.

So, drawing sand that has come to rest as o, the first unit of sand simply falls straight down and then stops:
```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
......o.#.
#########.
```
The second unit of sand then falls straight down, lands on the first one, and then comes to rest to its left:
```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
.....oo.#.
#########.
```
After a total of five units of sand have come to rest, they form this pattern:
```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
......o.#.
....oooo#.
#########.
```
After a total of 22 units of sand:
```
......+...
..........
......o...
.....ooo..
....#ooo##
....#ooo#.
..###ooo#.
....oooo#.
...ooooo#.
#########.
```
Finally, only two more units of sand can possibly come to rest:
```
......+...
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.
```
Once all 24 units of sand shown above have come to rest, all further sand flows out the bottom, falling into the endless void. Just for fun, the path any new sand takes before falling forever is shown here with ~:
```
.......+...
.......~...
......~o...
.....~ooo..
....~#ooo##
...~o#ooo#.
..~###ooo#.
..~..oooo#.
.~o.ooooo#.
~#########.
~..........
~..........
~..........
```
Using your scan, simulate the falling sand. How many units of sand come to rest before sand starts flowing into the abyss below?


In [114]:
f = open("input.txt", "r")
lines = f.read().splitlines()

#lines = [
#"498,4 -> 498,6 -> 496,6",
#"503,4 -> 502,4 -> 502,9 -> 494,9"
#]

def parse_input(lines):
    walls = []
    for line in lines:
        wall = []
        words = line.split(" -> ")
        for word in words:
            coords = word.split(",")
            wall.append([int(coords[0]),int(coords[1])])
        walls.append(wall)    
    return walls
        
def find_grid_size(walls):
    min_x, max_x = 500, 500
    min_y, max_y = 0, 0
    
    for wall in walls:
        for point in wall:
            if point[0] > max_x:
                max_x = point[0]
            elif point[0] < min_x:
                min_x = point[0]

            if point[1] > max_y:
                max_y = point[1]
            elif point[1] < min_y:
                min_y = point[1]

    return min_x, max_x, min_y, max_y


class Grid:
    def __init__(self, walls, floor = False, extend = 30):
        self.min_x, self.max_x, self.min_y, self.max_y = find_grid_size(walls)
        
        if floor:
            self.max_y += 2  # Add room for floor
            self.min_x -= extend # Extend width
            self.max_x += extend # Extend width
        
        self.width = self.max_x - self.min_x + 1
        self.height = self.max_y - self.min_y + 1
        
        self.grid = [ ["."]*self.width for i in range(self.height) ]
        print("Width", self.width, "Height", self.height)
        self.__fill_grid__(walls)
        
        if floor:
            self.__draw_wall([self.min_x, self.max_y], [self.max_x, self.max_y])
        
        # Starting point for sand
        self.sand = (500, 0)
        self[500, 0] = "+"
        self.rested_sand = 0
    
    def print(self):
        for line in self.grid:
            print(line)
    
    # Key is a tuple of (X, Y) coords
    def __getitem__(self, key):
        true_x = key[0] - self.min_x 
        true_y = key[1] - self.min_y
        
        if true_x < 0 or key[0] > self.max_x or true_y < 0 or key[1] > self.max_y:
            return None
        else:
            return self.grid[true_y][true_x]
        
    def __setitem__(self, key, value):
        true_x = key[0] - self.min_x 
        true_y = key[1] - self.min_y
        
        if true_x < 0 or key[0] > self.max_x or true_y < 0 or key[1] > self.max_y:
            print("Can't assign! Out of bounds at", key)
        else:
            self.grid[true_y][true_x] = value
    
    def __fill_grid__(self, walls):
        for wall in walls:
            for i in range(len(wall) - 1):
                self.__draw_wall(wall[i], wall[i+1])
    
    def __draw_wall(self, start, end):
        dx = end[0] - start[0]
        dy = end[1] - start[1]
        length = int(max(abs(dx), abs(dy))) + 1
        
        if dx > 0:
            dx = 1
        elif dx < 0:
            dx = -1
        if dy > 0:
            dy = 1
        elif dy < 0:
            dy = -1
        
        for i in range(length):
            px = start[0] + i*dx
            py = start[1] + i*dy
            self[px, py] = "#" # Set wall
            
    def add_sand_block(self):
        pos = self.sand
        
        can_fall = True
        while(can_fall):
            
            moves = [
                (pos[0], pos[1] + 1), # DOWN
                (pos[0] - 1, pos[1] + 1), # DOWN LEFT
                (pos[0] + 1, pos[1] + 1), # DOWN RIGHT
            ]
            
            moved = False
            
            for move in moves:
                if self[move] == None:
                    print("Sand went out of bounds!")
                    return True
                elif self[move] == ".":
                    pos = move
                    moved = True
                    break
            
            if not moved:
                can_fall = False
                self.rested_sand += 1
                self[pos] = "O"
                if pos == self.sand:
                    print("Sand rested at source!")
                    return True
    
    def add_all_sand(self):
        sand_out_of_bounds = False
        while not sand_out_of_bounds:
            if self.add_sand_block():
                sand_out_of_bounds = True
        
        print("Total rested sand: ", self.rested_sand)

In [105]:
# Part 1
walls = parse_input(lines)
grid = Grid(walls)
grid.add_all_sand()

Width 10 Height 10
Sand went out of bounds!
Total rested sand:  24


## --- Part Two ---

You realize you misread the scan. There isn't an endless void at the bottom of the scan - there's floor, and you're standing on it!

You don't have time to scan the floor, so assume the floor is an infinite horizontal line with a y coordinate equal to two plus the highest y coordinate of any point in your scan.

In the example above, the highest y coordinate of any point is 9, and so the floor is at y=11. (This is as if your scan contained one extra rock path like -infinity,11 -> infinity,11.) With the added floor, the example above now looks like this:
```
        ...........+........
        ....................
        ....................
        ....................
        .........#...##.....
        .........#...#......
        .......###...#......
        .............#......
        .............#......
        .....#########......
        ....................
<-- etc #################### etc -->
```
To find somewhere safe to stand, you'll need to simulate falling sand until a unit of sand comes to rest at 500,0, blocking the source entirely and stopping the flow of sand into the cave. In the example above, the situation finally looks like this after 93 units of sand come to rest:
```
............o............
...........ooo...........
..........ooooo..........
.........ooooooo.........
........oo#ooo##o........
.......ooo#ooo#ooo.......
......oo###ooo#oooo......
.....oooo.oooo#ooooo.....
....oooooooooo#oooooo....
...ooo#########ooooooo...
..ooooo.......ooooooooo..
#########################
```
Using your scan, simulate the falling sand until the source of the sand becomes blocked. How many units of sand come to rest?

In [117]:
# Part 2
walls = parse_input(lines)
grid = Grid(walls, True, 200)
grid.add_all_sand()

Width 459 Height 172
Sand rested at source!
Total rested sand:  27551


In [118]:
# Not too happy with my part 2, but it works :)